In [18]:
import pdfplumber as pdfp
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

month = '22-07'
pages = []

###################################################################################################################
# loading the data
###################################################################################################################

# open pdf statement, read each page and extract the text line by line, save results
with pdfp.open(f'Credit Card Statements/{month}.pdf') as pdf:

    for page in pdf.pages:
        pages.append(page.extract_text().split('\n'))

    for i, page in enumerate(pages):
        if i == 0:
            df = pd.DataFrame(page, columns=['text'])
        else:
            new_df = pd.DataFrame(page, columns=['text'])
            df = pd.concat([df, new_df])

###################################################################################################################
# some data cleaning
###################################################################################################################

# drop null values
df = df.dropna()

# select only lines that have a date associated with them
df = df.loc[df['text'].str[:3].str.contains('JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC'), :]

# split the date, description and amount out into their own columns
df['date'] = df['text'].str.split(' ').str[0]
df['place'] = df['text'].str.split(' ').str[2]
df['amount'] = df['text'].str.split(' ').str[3]

# reset the index and drop unnecessary columns
df = df.reset_index().drop(['index', 'text'], axis=1)
df = df.rename(columns={
    'date' : 'Date', 
    'place' : 'Place', 
    'amount' : 'Amount'
})

# touch up the amount column so that it will be formatted as a number in google sheet
df['Amount'] = df['Amount'].str.replace(',', '').str.replace('\$', '')

# drop rows with recurring charges, or entries which are recurring payments
df = df.loc[~df['Place'].str.lower().str.contains('spotify|fido|payment-thankyou|tsiinternet'), :]

###################################################################################################################
# set up the keywords which will be used to group the charges
###################################################################################################################

on_the_go_coffee = 'timhortons'\
                   '|starbuck'\
                   '|coffee'\
                   '|timothy'\
                   '|balzac'\
                   '|madawaskacoffee'\
                   '|secondcup'

beer_and_weed = 'lcbo'\
                '|beerstore'\
                '|oneplant'

take_out = 'subway'\
           '|domino'\
           '|a&w'\
           '|zoup'\
           '|amazing'\
           '|maestro'\
           '|emily'\
           '|vipei'\
           '|bigbrother'\
           '|booster'\
           '|freshly'\
           '|northwinds'\
           '|jerk'\
           '|milkylane'\
           '|burrito'\
           '|jusdanfoods'\
           '|cornwall'\
           '|bastard'\
           '|rolltation'\
           '|shawarma'\
           '|carleton'\
           '|doordash'\
           '|mcdonald'\
           '|caesar'

bars_and_restaurants = 'jackastor'\
                       '|portly'\
                       '|oldestone'\
                       '|magwyer'\
                       '|kelsey'\
                       '|chuuk'\
                       '|lacarnita'\
                       '|smitty'\
                       '|milestone'\
                       '|wildwing'\
                       '|popeyes'\
                       '|prenup'\
                       '|moose'\
                       '|sabai'\
                       '|borealis'\
                       '|thepint'\
                       '|chicago'\
                       '|spaghetti'\
                       '|bmofield'\
                       '|yummykorean'\
                       '|eggsmart'\
                       '|aokcraft'\
                       '|legendsmusic'\
                       '|aramark'

clothing = 'zara'\
           '|h&m'\
           '|aeo'\
           '|softmoc'\
           '|vans'\
           '|jack&jones'\
           '|oldnavy'\
           '|sportchek'

grocery = 'rcss'\
          '|freshco'\
          '|wal-mart'\
          '|nofrills'\
          '|loblaws'\
          '|zehrs'\
          '|metro'\
          '|foodbasics'

gas = 'shell'\
      '|petro'\
      '|macewen barrys'\
      '|pioneer'

###################################################################################################################
# assert that we're not double counting any charges by placing them in multiple categories
###################################################################################################################

cats = {
    'on_the_go_coffee' : on_the_go_coffee,
    'take_out' : take_out,
    'bars_and_restaurants' : bars_and_restaurants,
    'clothing' : clothing,
    'grocery' : grocery,
    'gas' : gas
}

for cat1 in cats:
    
    for cat2 in cats:
        
        if cat1 == cat2:
            continue
        else:
            for merchant in cats[cat1].split('|'):
                assert merchant not in cats[cat2].split('|'), \
                f'{cat1} contains duplicate values with {cat2}, duplicate value: {merchant}'

                
###################################################################################################################
### show the user what is in each category
###################################################################################################################

for i, cat in enumerate(cats):
    
    if i != 0:
        print('\n')
        
    print(f'{cat}: \n')
    for item in cats[cat].split('|'):
        
        print(f'\t{item}')
        
    input('Press enter to continue')

on_the_go_coffee: 

	timhortons
	starbuck
	coffee
	timothy
	balzac
	madawaskacoffee
	secondcup
Press enter to continue


take_out: 

	subway
	domino
	a&w
	zoup
	amazing
	maestro
	emily
	vipei
	bigbrother
	booster
	freshly
	northwinds
	jerk
	milkylane
	burrito
	jusdanfoods
	cornwall
	bastard
	rolltation
	shawarma
	carleton
	doordash
	mcdonald
	caesar
Press enter to continue


bars_and_restaurants: 

	jackastor
	portly
	oldestone
	magwyer
	kelsey
	chuuk
	lacarnita
	smitty
	milestone
	wildwing
	popeyes
	prenup
	moose
	sabai
	borealis
	thepint
	chicago
	spaghetti
	bmofield
	yummykorean
	eggsmart
	aokcraft
	legendsmusic
	aramark
Press enter to continue


clothing: 

	zara
	h&m
	aeo
	softmoc
	vans
	jack&jones
	oldnavy
	sportchek
Press enter to continue


grocery: 

	rcss
	freshco
	wal-mart
	nofrills
	loblaws
	zehrs
	metro
	foodbasics
Press enter to continue


gas: 

	shell
	petro
	macewen barrys
	pioneer
Press enter to continue


In [117]:
final_df = pd.DataFrame(columns=cat_names)

for i, cat_name in enumerate(cat_names):
    final_df.loc[0, cat_name] = df.loc[df['Place'].str.lower().str.contains(cats[i]), 'Amount'].astype('float64').sum()
final_df

,on_the_go_coffee,take_out,bars_and_restaurants,clothing,grocery,gas
0,100.75,124.87,72.68,0.0,271.9,61.79


In [118]:
full_list = on_the_go_coffee + '|' + beer_and_weed + '|' + take_out + '|' + bars_and_restaurants + '|' + clothing + '|' + grocery + '|' + gas
tmp = df.loc[~df['Place'].str.lower().str.contains(full_list), ['Place', 'Amount']]
tmp

,Place,Amount
1,BESTSELLERECOMCANADAINMONTREALQC,276.85
3,THEHOMEDEPOT#7166CAMBRIDGEON,30.48
11,NIKUFARMSTORONTOON,3.55
15,APPLE.COM/BILL866-712-7753ON,1.46
18,BESTBUY#943SCARBOROUGHON,56.49
21,THEHOMEDEPOT#7107TORONTOON,16.46
23,PRESTOAUTLTORONTOON,20.00
25,JAZZCUTHAIRSALONSCARBOROUGHON,19.49
27,PIANOTE.COMABBOTSFORDSOBC,44.42
28,CINEPLEXENTERTAINMENT416-323-6600ON,27.10


In [109]:
full_list

'timhortons|starbuck|coffee|timothy|balzac|madawaskacoffee|secondcup|subway|domino|a&w|zoup|amazing|maestro|emily|vipei|bigbrother|booster|freshly|northwinds|jerk|milkylane|burrito|jusdanfoods|cornwall|bastard|rolltation|shawarma|carleton|doordash|mcdonald|caesar|jackastor|portly|oldestone|magwyer|kelsey|chuuk|lacarnita|smitty|milestone|wildwing|popeyes|prenup|moose|sabai|borealis|thepint|chicago|spaghetti|bmofield|yummykorean|eggsmart|aokcraft|legendsmusic|aramark|zara|h&m|aeo|softmoc|vans|jack&jones|oldnavy|sportchek|rcss|freshco|wal-mart|nofrills|loblaws|zehrs|metro|foodbasics|shell|petro|macewen barrys|pioneer'

In [108]:
tmp[4].lower()

'thebeerstore#4162cambridgeon'

In [ ]:
df.loc[df['Place'].str.lower().str.contains()]

In [33]:
# build the categories
df['onTheGoCoffee'] = \
df.loc[df['Place'].str.lower().str.contains(onTheGoCoffee), 'Amount']\
                  .astype('float64').sum()

df['beerAndWeed'] = \
df.loc[df['Place'].str.lower().str.contains(beerAndWeed), 'Amount']\
                  .astype('float64').sum()

df['takeOutMeals'] = \
df.loc[df['Place'].str.lower().str.contains(takeOutAndMeals), 'Amount']\
                  .astype('float64').sum()

df['grocery'] = \
df.loc[df['Place'].str.lower().str.contains(grocery), 'Amount']\
                  .astype('float64').sum()

df['gas'] = \
df.loc[df['Place'].str.lower().str.contains(gas), 'Amount']\
                  .astype('float64').sum()

df['mallPurchases'] = \
df.loc[df['Place'].str.lower().str.contains(mallPurchases), 'Amount']\
                  .astype('float64').sum()

df['theRest'] = df.loc[(~df['Place'].str.lower().str.contains(gas)) &
                       (~df['Place'].str.lower().str.contains(grocery)) &
                       (~df['Place'].str.lower().str.contains(takeOutAndMeals)) &
                       (~df['Place'].str.lower().str.contains(beerAndWeed)) &
                       (~df['Place'].str.lower().str.contains(onTheGoCoffee)) &
                       (~df['Place'].str.lower().str.contains(mallPurchases)) &
                       (~df['Place'].str.lower().str.contains(stuffToIgnoreInTheRest)), 'Amount']\
                    .astype('float64').sum()

# df.to_csv(f'Parsed Credit Card Statements/{month}.csv', index=False)
df

,Date,Place,Amount,onTheGoCoffee,beerAndWeed,takeOutMeals,grocery,gas,mallPurchases,theRest
0,JUN26,TIMHORTONS#2924CAMBRIDGEON,4.54,96.6,148.85,197.55,271.9,61.79,0.0,533.61
1,JUN27,BESTSELLERECOMCANADAINMONTREALQC,276.85,96.6,148.85,197.55,271.9,61.79,0.0,533.61
3,JUN27,THEHOMEDEPOT#7166CAMBRIDGEON,30.48,96.6,148.85,197.55,271.9,61.79,0.0,533.61
4,JUN27,THEBEERSTORE#4162CAMBRIDGEON,148.85,96.6,148.85,197.55,271.9,61.79,0.0,533.61
5,JUN27,MCDONALD'S#25010Q04CAMBRIDGEON,33.18,96.6,148.85,197.55,271.9,61.79,0.0,533.61
6,JUN28,LITTLECAESARS#4749-0002CAMBRIDGEON,57.05,96.6,148.85,197.55,271.9,61.79,0.0,533.61
7,JUN30,TSIINTERNETCHATHAMON,144.14,96.6,148.85,197.55,271.9,61.79,0.0,533.61
9,JUL02,TIMHORTONS#2642PICKERINGON,9.04,96.6,148.85,197.55,271.9,61.79,0.0,533.61
10,JUL03,TIMHORTONS#2924CAMBRIDGEON,19.51,96.6,148.85,197.55,271.9,61.79,0.0,533.61
11,JUL04,NIKUFARMSTORONTOON,3.55,96.6,148.85,197.55,271.9,61.79,0.0,533.61


In [5]:
month = '22-07'
pages = []

with pdfp.open(f'Credit Card Statements/{month}.pdf') as pdf:

    for page in pdf.pages:
        pages.append(page.extract_text().split('\n'))

    for i, page in enumerate(pages):
        if i == 0:
            df = pd.DataFrame(page, columns=['text'])
        else:
            new_df = pd.DataFrame(page, columns=['text'])
            df = pd.concat([df, new_df])

df = df.dropna()
df = df.loc[df['text'].str[:3].str.contains('JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC'), :]
df = df.loc[~df['text'].str.lower().str.contains('spotify|fido'), :]
df['date'] = df['text'].str.split(' ').str[0]
df['place'] = df['text'].str.split(' ').str[2]
df['amount'] = df['text'].str.split(' ').str[3]
df = df.reset_index().drop('index', axis=1)
df = df.drop('text', axis=1)
#df = df.loc[df['amount'].str[0] != '-', :]
df = df.loc[~df['place'].str.lower().str.contains('payment-thankyou'), :]
df = df.rename(columns={'date' : 'Date', 'place' : 'Place', 'amount' : 'Amount'})

df['Amount'] = df['Amount'].str.replace(',', '')
df['Amount'] = df['Amount'].str.replace('\$', '')

# categorize the charges
onTheGoCoffee = 'timhortons|starbuck|coffee|timothy|balzac|madawaskacoffee'
beerAndWeed = 'lcbo|beerstore|oneplant'
takeOutAndMeals = 'subway|domino|a&w|zoup|amazing|jackastor|portly|oldestone'\
                                            '|magwyer|kelsey|maestro|chuuk|lacarnita|emily|vipei|bigbrother'\
                                            '|smitty|pizza|sushi|booster|freshly|northwinds|jerk'\
                                            '|milkylane|burrito|milestone|wildwing|popeyes|jusdanfoods'\
                                            '|prenup|moose|cornwall|bastard|sabai|borealis|thepint'\
                                            '|rolltation|chicago|spaghetti|shawarma|lounge|carleton'\
                                            '|doordash|bmofield|yummykorean|eggsmart|legendsmusic|aokcraft|aramark'\
                                            '|mcdonald|caesar'
mallPurchases = 'zara|h&m|aeo|softmoc|vans|jack&jones|oldnavy|sportchek'
grocery = 'rcss|freshco|wal-mart|nofrills|loblaws|zehrs|metro|foodbasics'
gas = 'shell|petro|macewen barrys|pioneer'#|esso'
stuffToIgnoreInTheRest = 'homehealthcare|jasonsmith|fido|spotify|internet|bestseller'


# build the categories
df['onTheGoCoffee'] = \
df.loc[df['Place'].str.lower().str.contains(onTheGoCoffee), 'Amount']\
                  .astype('float64').sum()

df['beerAndWeed'] = \
df.loc[df['Place'].str.lower().str.contains(beerAndWeed), 'Amount']\
                  .astype('float64').sum()

df['takeOutMeals'] = \
df.loc[df['Place'].str.lower().str.contains(takeOutAndMeals), 'Amount']\
                  .astype('float64').sum()

df['grocery'] = \
df.loc[df['Place'].str.lower().str.contains(grocery), 'Amount']\
                  .astype('float64').sum()

df['gas'] = \
df.loc[df['Place'].str.lower().str.contains(gas), 'Amount']\
                  .astype('float64').sum()

df['mallPurchases'] = \
df.loc[df['Place'].str.lower().str.contains(mallPurchases), 'Amount']\
                  .astype('float64').sum()

df['theRest'] = df.loc[(~df['Place'].str.lower().str.contains(gas)) &
                       (~df['Place'].str.lower().str.contains(grocery)) &
                       (~df['Place'].str.lower().str.contains(takeOutAndMeals)) &
                       (~df['Place'].str.lower().str.contains(beerAndWeed)) &
                       (~df['Place'].str.lower().str.contains(onTheGoCoffee)) &
                       (~df['Place'].str.lower().str.contains(mallPurchases)) &
                       (~df['Place'].str.lower().str.contains(stuffToIgnoreInTheRest)), 'Amount']\
                    .astype('float64').sum()

df.to_csv(f'Parsed Credit Card Statements/{month}.csv', index=False)
df

,Date,Place,Amount,onTheGoCoffee,beerAndWeed,takeOutMeals,grocery,gas,mallPurchases,theRest
0,JUN26,TIMHORTONS#2924CAMBRIDGEON,4.54,96.6,148.85,197.55,271.9,61.79,0.0,533.61
1,JUN27,BESTSELLERECOMCANADAINMONTREALQC,276.85,96.6,148.85,197.55,271.9,61.79,0.0,533.61
2,JUN27,THEHOMEDEPOT#7166CAMBRIDGEON,30.48,96.6,148.85,197.55,271.9,61.79,0.0,533.61
3,JUN27,THEBEERSTORE#4162CAMBRIDGEON,148.85,96.6,148.85,197.55,271.9,61.79,0.0,533.61
4,JUN27,MCDONALD'S#25010Q04CAMBRIDGEON,33.18,96.6,148.85,197.55,271.9,61.79,0.0,533.61
5,JUN28,LITTLECAESARS#4749-0002CAMBRIDGEON,57.05,96.6,148.85,197.55,271.9,61.79,0.0,533.61
6,JUN30,TSIINTERNETCHATHAMON,144.14,96.6,148.85,197.55,271.9,61.79,0.0,533.61
8,JUL02,TIMHORTONS#2642PICKERINGON,9.04,96.6,148.85,197.55,271.9,61.79,0.0,533.61
9,JUL03,TIMHORTONS#2924CAMBRIDGEON,19.51,96.6,148.85,197.55,271.9,61.79,0.0,533.61
10,JUL04,NIKUFARMSTORONTOON,3.55,96.6,148.85,197.55,271.9,61.79,0.0,533.61
